## Using ReAct Framework to analyse a stock.
 - Get the latest stock data from a data source.
 - Get the stock symbol, its financials
 - Get the news data headlines from the internet.

* We will use Llama3.1 to do this.

In [1]:
import json
import regex as re
import sys
import os
from typing import Optional

from llama_index.llms.ollama import Ollama
from llama_index.llms.groq import Groq
from llama_index.core import Settings
# from llama_index.tools.google import GoogleSearchToolSpec
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
# from llama_index.tools.brave_search import BraveSearchToolSpec
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.core import PromptTemplate

from dotenv import load_dotenv

# finance tool
import yfinance as yf
import datetime as dt

# get the prompts
sys.path.append(os.path.dirname(os.path.abspath("../stock_analyser_react.ipynb")))
from prompts.react_agent_prompts import LLAMAINDEX_REACT_BASE_PROMPT, STOCK_ANALYSER_PROMPT


# Load the env variables
load_dotenv("../secrets/local.env")

True

In [2]:
# LLM ollama
# llm = Ollama(model='llama3.1:latest', request_timeout=120.0, temperature=0.01)
# Settings.llm = llm

# LLM Groq
llm = Groq(model="llama-3.1-70b-versatile")
Settings.llm = llm

In [3]:
stream_response = llm.stream_complete("What is the stock symbol for Bajaj Finance?")

for t in stream_response:
    print(t.delta, end="")


# Note: how the model is hallucinating

The stock symbol for Bajaj Finance is BAJFINANCE.

In [4]:
# define the tools


def multiply(a: int, b: int) -> int:
    """Multiplies two integers and returns the result integer.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The product of a and b.
    """
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer.

    Args:
        a (int): The first integer.
        b (int): The second integer.
    
    Returns:
        int: The sum of a and b
    """
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer
    
    Args:
        a (int): The first integer.
        b (int): The second integer.
    
    Returns:
        int: The sum of a and b
    """
    return a - b


def divide(a: int, b: int) -> int:
    """Divides two integers and returns the result integer.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The quotient of a divided by b.
    """
    return a // b


# yahoo finance financials fetcher
def get_financial_statements(ticker: str) -> str:
    """Fetches the financial statements data from yahoo finance.

    Args:
        ticker (str): The ticker symbol of the company to fetch information for.

    Returns:
        str: A string containing the financial statement data in a formatted way.
    """

    if "." in ticker:
        ticker = ticker.split(".")[0]

    ticker += ".NS"
    company = yf.Ticker(ticker)
    balance_sheet = company.balance_sheet
    if balance_sheet.shape[-1] > 3:
        balance_sheet = balance_sheet.iloc[:, :3]

    balance_sheet = balance_sheet.dropna(how="any")
    balance_sheet = "\n" + balance_sheet.to_string()

    return balance_sheet


# get stock info and recommendations summary
def get_stockinfo(ticker: str) -> str:
    """Provides the detailed financial and general information about the stock ticker.
        Also provides a table for analyst recommendations for the ticker.

    Args:
        ticker (str): The ticker symbol of the company to fetch information for.
    
    Returns:
        str: A string containing the stock info and a summary of recommendations.
    """

    if "." in ticker:
        ticker = ticker.split(".")[0]

    ticker += ".NS"

    company = yf.Ticker(ticker)

    stock_info = company.info
    try:
        recommendations_summary = company.recommendations_summary
    except:
        recommendations_summary = ""

    # TODO: add units and convert to easily understandable units

    include_info = ["industry", "sector", "longBuisnessSummary", "previousClose", "dividendRate", "dividentYield", "beta", "forwardPE", "volume", "marketCap", "fiftyTwoWeekLow", "fiftyTwoWeekHigh", "currency", "bookValue", "priceToBook", "earningsQuarterlyGrowth", "trailingEps", "forwardEps", "52WeekChange", "totalCashPerShare", "ebidta", "totabDebt", "totalCashPerShare", "debtToEquity", "revenuePerShare", "earningsGrowth", "revenueGrowth", "grossMargins", "ebidtaMargins", "operatingMargins", ]
    response = "#### Stock info:\n"

    for key, val in stock_info.items():
        if key in include_info:
            if re.search("(Growth|Margin|Change)", key):
                response += f"{key}: {round(float(val) * 100, 3)} %\n"
            elif "marketCap" in key:
                response += f"{key}: {round(int(val) * 1e-7, 2)} Cr.\n"
            else:
                response += f"{key}: {val}\n"

    response += "\n#### Analyst Recommendations:\n"
    response += f"\n{recommendations_summary}"

    return response


multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
sub_tool = FunctionTool.from_defaults(fn=subtract)
div_tool = FunctionTool.from_defaults(fn=divide)

# financial sheets
fin_statements = FunctionTool.from_defaults(fn=get_financial_statements)

# stock information and the analyst recommendations
stockinfo = FunctionTool.from_defaults(fn=get_stockinfo)

ddg_tools = DuckDuckGoSearchToolSpec().to_tool_list()
ddg_full_search_tool = ddg_tools[1]
ddg_instant_search_tool = ddg_tools[0]

In [5]:
multiply.__annotations__

{'a': int, 'b': int, 'return': int}

In [6]:
ddg_full_search_tool("bajaj finance ticker NSE/BSE")

ToolOutput(content='[{\'title\': \'Bajaj Finance Limited Share Price Today, Stock Price, Live NSE News ...\', \'href\': \'https://www.nseindia.com/get-quotes/equity?symbol=BAJFINANCE\', \'body\': \'Bajaj Finance Limited Share Price Today, Live NSE Stock Price: Get the latest Bajaj Finance Limited news, company updates, quotes, offers, annual financial reports, graph, volumes, 52 week high low, buy sell tips, balance sheet, historical charts, market performance, capitalisation, dividends, volume, profit and loss account, research, results and more details at NSE India.\'}, {\'title\': \'Bajaj Finance Ltd (BAJFINANCE) Stock Price & News - Google\', \'href\': \'https://www.google.com/finance/quote/BAJFINANCE:NSE\', \'body\': \'Get the latest Bajaj Finance Ltd (BAJFINANCE) real-time quote, historical performance, charts, and other financial information to help you make more informed trading and investment decisions.\'}, {\'title\': \'Bajaj Finance Limited Live Stock Price , Bajfinance Live

In [7]:
ddg_instant_search_tool.metadata

ToolMetadata(description='duckduckgo_instant_search(query: str) -> List[Dict]\n\n        Make a query to DuckDuckGo api to receive an instant answer.\n\n        Args:\n            query (str): The query to be passed to DuckDuckGo.\n        ', name='duckduckgo_instant_search', fn_schema=<class 'llama_index.core.tools.utils.duckduckgo_instant_search'>, return_direct=False)

In [8]:
# Define a ReactAgent
agent = ReActAgent.from_tools(tools=[ddg_full_search_tool, fin_statements, stockinfo],
                   llm=llm,
                   verbose=True,
                   max_iterations=25,
                   context="You are a genius Research Analyst with a Phd in Economics. You analyse equity markets very well and are proficient in creating detailed analysis reports. You do not provide recommendations only analyse company stocks.")


# have a look at the agent prompt
agent.get_prompts()

{'agent_worker:system_prompt': PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['tool_desc', 'context', 'tool_names'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.\n\n## Tools\n\nYou have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools to complete each subtask.\n\nYou have access to the following tools:\n{tool_desc}\n\nHere is some context to help you answer the question and plan:\n{context}\n\n\n## Output Format\n\nPlease answer in the same language as the question and use the following format:\n\n```\nThought: The current language of the user is: (user\'s language). I need to use a tool to help me answer the 

In [9]:
# assign the prompt to the agent
stock_analyser_prompt = PromptTemplate(template=STOCK_ANALYSER_PROMPT)
agent.update_prompts(
    {'agent_worker:system_prompt': stock_analyser_prompt}
)

In [14]:
response = agent.chat("Analyse SAIL")

> Running step dd14a99e-54b8-47b5-b2fd-5e08b0a44f46. Step input: Analyse SAIL
Thought: I need to extract the company name from the input query.
Action: None
Action Input: {'query': 'SAIL NSE/BSE ticker', 'region': 'in'}
Observation: Error: No such tool named `None`.
> Running step b5f2496c-255d-4ec5-830f-66473d92c4f4. Step input: None
Thought: I need to use the duckduckgo_full_search tool to find the ticker symbol for SAIL.
Action: duckduckgo_full_search
Action Input: {'query': 'SAIL NSE/BSE ticker', 'region': 'in', 'max_results': 1}
Observation: Error: https://links.duckduckgo.com/d.js RuntimeError: error sending request for url (https://links.duckduckgo.com/d.js?q=SAIL+NSE%2FBSE+ticker&kl=in&l=in&p=&s=0&df=&vqd=4-23823199885436030106791529423505061141&bing_market=-IN&ex=-1): operation timed out

Caused by:
    operation timed out
> Running step fcb8f100-6081-49f6-b1ee-01bc45490463. Step input: None
Thought: The previous attempt to find the ticker symbol for SAIL timed out. I will try

In [10]:
response

AgentChatResponse(response="Based on the analysis, I would recommend buying Bajaj Finance stock, but with caution due to the high price-to-book ratio and debt-to-equity ratio. The company's strong financial position, high level of profitability, and strong potential for future growth make it an attractive investment opportunity. However, investors should be aware of the potential risks associated with the high debt-to-equity ratio and the relatively high price-to-book ratio. I am just an AI Research analyst assistant. I am not a financial advisor. I don't provide investment advice or recommendations.", sources=[ToolOutput(content='[{\'title\': \'Bajaj Finance Limited Share Price Today, Stock Price, Live NSE News ...\', \'href\': \'https://www.nseindia.com/get-quotes/equity?symbol=BAJFINANCE\', \'body\': \'Bajaj Finance Limited Share Price Today, Live NSE Stock Price: Get the latest Bajaj Finance Limited news, company updates, quotes, offers, annual financial reports, graph, volumes, 52